In [1]:
from fastcore.all import *
from fastai.vision.all import *
import time
import timm
import sys
from kaggle import api
import gc

/home/gegg00/mambaforge/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models_to_load = ['convnext_base_in22ft1k',  'convnext_large_384_in22ft1k',  'convnext_large_in22ft1k',  'convnext_large_in22k']
sizes = [[(240, 320), 'squish', 256, 'zeros', 2], [(480, 640), 'squish', 256, 'border',4], [480, 'crop', 128, 'reflection',4], [(240,320), 'squish', 256, 'zeros',2]]

In [3]:
models_to_load, sizes

(['convnext_base_in22ft1k',
  'convnext_large_384_in22ft1k',
  'convnext_large_in22ft1k',
  'convnext_large_in22k'],
 [[(240, 320), 'squish', 256, 'zeros', 2],
  [(480, 640), 'squish', 256, 'border', 4],
  [480, 'crop', 128, 'reflection', 4],
  [(240, 320), 'squish', 256, 'zeros'],
  2])

In [4]:
path = Path('.')
trn_path = Path(path/"train_images")
tst_files = get_image_files(path/'test_images').sorted()

In [5]:
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [6]:
def train(arch, item=Resize(480, method='squish'), aug= aug_transforms(size=480, min_scale=0.75), accum=1, finetune=True, epochs=12):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item,
        batch_tfms=aug, bs=64//accum)
    cbs = GradientAccumulation(accum) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)
    return learn

In [7]:
idx = 0
arch = models_to_load[idx]
rz, rzm, sz, pd, accum= sizes[idx]
_item = Resize(rz, method=rzm)
_trans = aug_transforms(size=sz, pad_mode=pd, min_scale=0.75)
train(models_to_load[0], item=_item, aug=_trans, accum=2, finetune=False, epochs=1)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,nan,nan,0.946180,03:48


<function __main__.report_gpu()>

In [10]:
idx = 1
arch = models_to_load[idx]
rz, rzm, sz, pd, accum= sizes[idx]
_item = Resize(rz, method=rzm)
_trans = aug_transforms(size=sz, pad_mode=pd, min_scale=0.75)
train(models_to_load[0], item=_item, aug=_trans, accum=2, finetune=False, epochs=1)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,nan,nan,0.955310,03:53


GPU:0
process    1401168 uses     8255.000 MB GPU memory


In [11]:
idx = 2
arch = models_to_load[idx]
rz, rzm, sz, pd, accum= sizes[idx]
_item = Resize(rz, method=rzm)
_trans = aug_transforms(size=sz, pad_mode=pd, min_scale=0.75)
train(models_to_load[0], item=_item, aug=_trans, accum=2, finetune=False, epochs=1)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,nan,nan,0.948582,01:11


GPU:0
process    1401168 uses     4217.000 MB GPU memory


In [12]:
idx = 3
arch = models_to_load[idx]
rz, rzm, sz, pd, accum= sizes[idx]
_item = Resize(rz, method=rzm)
_trans = aug_transforms(size=sz, pad_mode=pd, min_scale=0.75)
train(models_to_load[0], item=_item, aug=_trans, accum=2, finetune=False, epochs=1)
report_gpu()

ValueError: not enough values to unpack (expected 5, got 4)

In [8]:
tta_res = []

for idx, arch in enumerate(models_to_load):
    rz, rzm, sz, pd, accum= sizes[idx]
    _item = Resize(rz, method=rzm)
    _trans = aug_transforms(size=sz, pad_mode=pd, min_scale=0.75)
    print('---',arch, rz, rzm, sz, pd)
    tta_res.append(train(arch,item=_item, aug=_trans, accum=accum , epochs=14))
    gc.collect()
    torch.cuda.empty_cache()

--- convnext_base_in22ft1k (240, 320) squish 256 zeros


epoch,train_loss,valid_loss,error_rate,time


KeyboardInterrupt: 